In [2]:
import numpy as np

import tensorflow_datasets as tfds
import tensorflow as tf

tfds.disable_progress_bar()


In [3]:
import matplotlib.pyplot as plt


def plot_graphs(history, metric):
    plt.plot(history.history[metric])
    plt.plot(history.history['val_'+metric], '')
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.legend([metric, 'val_'+metric])

In [4]:
# nhap du lieu
import collections
import pathlib

import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import utils
from tensorflow.keras.layers import TextVectorization

import tensorflow_datasets as tfds
import tensorflow_text as tf_text

In [8]:
# data_url = 'http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'

# dataset_dir = utils.get_file(
#     fname = 'imdb_reviews',
#     origin=data_url,
#     untar=True
# )

# dataset_dir = pathlib.Path(dataset_dir).parent

84140032/84125825 [==============================] - 167s 2us/step


In [10]:
dataset, info = tfds.load('imdb_reviews', with_info=True,as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

train_dataset.element_spec

Dataset imdb_reviews downloaded and prepared to C:\Users\lucky\tensorflow_datasets\imdb_reviews\plain_text\1.0.0. Subsequent calls will reuse this data.


(TensorSpec(shape=(), dtype=tf.string, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [11]:
dataset_dir

WindowsPath('C:/Users/lucky/.keras/datasets')

In [16]:
info

tfds.core.DatasetInfo(
    name='imdb_reviews',
    full_name='imdb_reviews/plain_text/1.0.0',
    description="""
    Large Movie Review Dataset. This is a dataset for binary sentiment
    classification containing substantially more data than previous benchmark
    datasets. We provide a set of 25,000 highly polar movie reviews for training,
    and 25,000 for testing. There is additional unlabeled data for use as well.
    """,
    config_description="""
    Plain text
    """,
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    data_path='C:\\Users\\lucky\\tensorflow_datasets\\imdb_reviews\\plain_text\\1.0.0',
    file_format=tfrecord,
    download_size=80.23 MiB,
    dataset_size=129.83 MiB,
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=int64, num_classes=2),
        'text': Text(shape=(), dtype=string),
    }),
    supervised_keys=('text', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=25000, num

In [12]:
for example, label in train_dataset.take(1):
    print('text: ', example.numpy())
    print('label: ', label.numpy())

text:  b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
label:  0


In [13]:
#tham so cho xao tron du lieu
BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [14]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [15]:
for example, label in train_dataset.take(1):
    print('texts: ', example.numpy()[:3])
    print()
    print('labels: ', label.numpy()[:3])

texts:  [b'Paris is the place to be to enjoy beautiful art and music, and to fall madly in love - as is the case in this film. Boy meets girl, they fall in love, but something stands in their way of eternal happiness, the classic story.<br /><br />The wonderful music of George Gerschwin complements the great dancing by Gene Kelly and Leslie Caron. "An American in Paris" is a humorous, light-hearted, loving film well worth watching.<br /><br />8/10<br /><br />'
 b"It's not really about gymnastics; swap out the occasional training montages and it could just as easily be about archery, or microbiology, or a booger-flicking tournament. Instead, like every other Rocky/Flashdance derivative that flooded the 80s market, it's about conquering adversity with stick-to-it-iveness, rendering all social/personal realities irrelevant by your lonesome - with love interest standing by of course. Ronald Reagan top to bottom, in short; so as a piece of cinema it's down to the details. Some of the actors

In [17]:
#tao bo ma hoa van ban
VOCAB_SIZE = 1000
encoder = tf.keras.layers.TextVectorization(max_tokens=VOCAB_SIZE)
encoder.adapt(train_dataset.map(lambda text, label: text))

In [18]:
vocab = np.array(encoder.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'the', 'and', 'a', 'of', 'to', 'is', 'in', 'it', 'i',
       'this', 'that', 'br', 'was', 'as', 'for', 'with', 'movie', 'but'],
      dtype='<U14')

In [19]:
encoded_example = encoder(example)[:3].numpy()
encoded_example

array([[  1,   7,   2, ...,   0,   0,   0],
       [ 30,  22,  63, ...,   0,   0,   0],
       [ 41, 163,  31, ...,   0,   0,   0]], dtype=int64)

In [20]:
for n in range(3):
    print("Original: ", example[n].numpy())
    print("Round-trip: ", " ".join(vocab[encoded_example[n]]))
    print()

Original:  b'Paris is the place to be to enjoy beautiful art and music, and to fall madly in love - as is the case in this film. Boy meets girl, they fall in love, but something stands in their way of eternal happiness, the classic story.<br /><br />The wonderful music of George Gerschwin complements the great dancing by Gene Kelly and Leslie Caron. "An American in Paris" is a humorous, light-hearted, loving film well worth watching.<br /><br />8/10<br /><br />'
Round-trip:  [UNK] is the place to be to enjoy beautiful art and music and to fall [UNK] in love as is the case in this film boy meets girl they fall in love but something [UNK] in their way of [UNK] [UNK] the classic [UNK] br the wonderful music of george [UNK] [UNK] the great [UNK] by [UNK] [UNK] and [UNK] [UNK] an american in [UNK] is a [UNK] [UNK] [UNK] film well worth [UNK] br [UNK] br                                                                                                                                            

In [21]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [22]:
print([layer.supports_masking for layer in model.layers])

[False, True, True, True, True]


In [23]:
# predict on a sample text without padding.

sample_text = ('The movie was cool. The animation and the graphics '
               'were out of this world. I would recommend this movie.')
predictions = model.predict(np.array([sample_text]))
print(predictions[0])

[0.01253613]


In [24]:
# predict on a sample text with padding

padding = "the " * 2000
predictions = model.predict(np.array([sample_text, padding]))
print(predictions[0])

[0.01253613]


In [25]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [26]:
#train model
history = model.fit(train_dataset, epochs=10,
                    validation_data=test_dataset,
                    validation_steps=30)

Epoch 1/10
391/391 [==============================] - 850s 2s/step - loss: 0.6424 - accuracy: 0.5700 - val_loss: 0.5105 - val_accuracy: 0.7245
Epoch 2/10
391/391 [==============================] - 779s 2s/step - loss: 0.4073 - accuracy: 0.8114 - val_loss: 0.3755 - val_accuracy: 0.8151
Epoch 3/10
391/391 [==============================] - 708s 2s/step - loss: 0.3401 - accuracy: 0.8501 - val_loss: 0.3390 - val_accuracy: 0.8458
Epoch 4/10
391/391 [==============================] - 720s 2s/step - loss: 0.3211 - accuracy: 0.8616 - val_loss: 0.3264 - val_accuracy: 0.8557
Epoch 5/10
391/391 [==============================] - 884s 2s/step - loss: 0.3131 - accuracy: 0.8664 - val_loss: 0.3301 - val_accuracy: 0.8526
Epoch 6/10
359/391 [==========================>...] - ETA: 1:01 - loss: 0.3063 - accuracy: 0.8679

CancelledError: Graph execution error:

Loop execution was cancelled.
	 [[{{node gradients/while_grad/while_grad/LoopCond/_94}}]]
	 [[Adam/gradients/PartitionedCall]] [Op:__inference_train_function_18760]

In [27]:
test_loss, test_acc = model.evaluate(test_dataset)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

389/391 [============================>.] - ETA: 0s - loss: 0.3294 - accuracy: 0.8388

ResourceExhaustedError: Graph execution error:

OOM when allocating tensor with shape[963,64,64] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[{{node ReverseV2}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.

	 [[sequential/bidirectional/backward_lstm/PartitionedCall]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_test_function_23795]

In [ ]:
plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plot_graphs(history, 'accuracy')
plt.ylim(None, 1)
plt.subplot(1, 2, 2)
plot_graphs(history, 'loss')
plt.ylim(0, None)

In [ ]:
sample_text = ('The movie was cool. The animation and the graphics '
               'were out of this world. I would recommend this movie.')
predictions = model.predict(np.array([sample_text]))

In [ ]:
# Xếp chồng hai hoặc nhiều lớp LSTM
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(len(encoder.get_vocabulary()), 64, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1)
])

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_dataset, epochs=10,
                    validation_data=test_dataset,
                    validation_steps=30)

In [ ]:
test_loss, test_acc = model.evaluate(test_dataset)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

In [ ]:
# predict on a sample text without padding.

sample_text = ('The movie was not good. The animation and the graphics '
               'were terrible. I would not recommend this movie.')
predictions = model.predict(np.array([sample_text]))
print(predictions)

In [ ]:
plt.figure(figsize=(16, 6))
plt.subplot(1, 2, 1)
plot_graphs(history, 'accuracy')
plt.subplot(1, 2, 2)
plot_graphs(history, 'loss')